In [33]:
import pandas as pd

# Load the Excel file
file_path = r"C:\Users\OMEN\OneDrive\Desktop\vaccine-schedule-data.xlsx"
xls = pd.ExcelFile(file_path)

# Load both sheets into DataFrames
df_data = pd.read_excel(xls, sheet_name="Data")
df_reference = pd.read_excel(xls, sheet_name="Reference")

# Display first few rows
df_data.head(), df_reference.head()


(  ISO_3_CODE COUNTRYNAME WHO_REGION    YEAR VACCINECODE  \
 0        ABW       Aruba       AMRO  2023.0  DTAPHIBIPV   
 1        ABW       Aruba       AMRO  2023.0  DTAPHIBIPV   
 2        ABW       Aruba       AMRO  2023.0  DTAPHIBIPV   
 3        ABW       Aruba       AMRO  2023.0  DTAPHIBIPV   
 4        ABW       Aruba       AMRO  2023.0     DTAPIPV   
 
                 VACCINE_DESCRIPTION  SCHEDULEROUNDS  TARGETPOP  \
 0  DTaP-Hib-IPV (acellular) vaccine             1.0        NaN   
 1  DTaP-Hib-IPV (acellular) vaccine             2.0        NaN   
 2  DTaP-Hib-IPV (acellular) vaccine             3.0        NaN   
 3  DTaP-Hib-IPV (acellular) vaccine             4.0    B_2YL_W   
 4      DTaP-IPV (acellular) vaccine             5.0  B_CHILD_W   
 
   TARGETPOP_DESCRIPTION   GEOAREA AGEADMINISTERED SOURCECOMMENT  
 0       General/routine  NATIONAL              M2           NaN  
 1       General/routine  NATIONAL              M4           NaN  
 2       General/routine  NATIONA

In [34]:
# Check missing values in TARGETPOP column
missing_targetpop = df_data["TARGETPOP"].isna().sum()
total_rows = len(df_data)

In [35]:
# Check unique values in TARGETPOP_DESCRIPTION and AGEADMINISTERED
unique_targetpop_desc = df_data["TARGETPOP_DESCRIPTION"].unique()
unique_ageadministered = df_data["AGEADMINISTERED"].unique()

missing_targetpop, total_rows, unique_targetpop_desc, unique_ageadministered


(np.int64(4258),
 8053,
 array(['General/routine', 'Risk group(s)', 'Travellers', 'Health workers',
        'HPV for females only', 'Adults', 'Pregnant women',
        'Catch-up adults', 'Catch-up children',
        'HPV administered to females and males', nan], dtype=object),
 array(['M2', 'M4', 'M6', 'M15', 'Y4', 'Y10', '1st contact', '+M1', '+M5',
        '+M6', 'M1', 'M3', 'M9', 'Y11', nan, 'M12', '>=Y1', 'B', 'W6',
        'W10', 'W14', 'M18', '+Y1', 'Y15-Y49', 'Y4-Y5', '<Y25', 'Y9-15',
        '>Y18', '>=M6', 'Y1', 'Y2', 'Y15', '+W4', '+M6-Y1', '>Y4', 'Y6',
        'Y13', '>Y9', 'M6-Y9', 'Y5', 'Y18', 'Y12', '>=Y65', 'M13', 'Y3',
        'M5', 'Y5-Y6', 'Y15-Y16', '>=Y50', 'M15-M18', '+M2', 'M6-M24',
        '>Y65', '+D7', '+D21', '+M6-M12', '+Y5', '1st Contact', 'W12',
        'W18', 'Y14', 'Y15-Y45', 'M6-Y2', 'Y3-Y5', 'Y4-Y6', 'W16',
        'Y26-Y30', 'Y36-Y40', 'Y46-Y50', 'Y56-Y60', 'Y66-Y70', 'Y76-Y80',
        'Y17', '>=Y7', 'Y9-Y13', 'M6-M35', '>=Y3', '>=Y2', 'Y25', '>=Y19',

In [36]:
# Check the most common TARGETPOP values for each TARGETPOP_DESCRIPTION
targetpop_mapping = df_data.groupby("TARGETPOP_DESCRIPTION")["TARGETPOP"].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

In [37]:
# Check the most common TARGETPOP values for each AGEADMINISTERED
age_mapping = df_data.groupby("AGEADMINISTERED")["TARGETPOP"].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

targetpop_mapping, age_mapping


(TARGETPOP_DESCRIPTION
 Adults                                       ADULTS
 Catch-up adults                           CATCHUP_A
 Catch-up children                         CATCHUP_C
 General/routine                           B_CHILD_W
 HPV administered to females and males          BOTH
 HPV for females only                         FEMALE
 Health workers                                   HW
 Pregnant women                                   PW
 Risk group(s)                            RISKGROUPS
 Travellers                               TRAVELLERS
 Name: TARGETPOP, dtype: object,
 AGEADMINISTERED
 +D14    RISKGROUPS
 +D21    RISKGROUPS
 +D28    RISKGROUPS
 +D3     RISKGROUPS
 +D30    RISKGROUPS
            ...    
 М2            None
 М3            None
 М4            None
 М4.5          None
 М6            None
 Name: TARGETPOP, Length: 425, dtype: object)

In [38]:
# Step 1: Fill missing TARGETPOP based on TARGETPOP_DESCRIPTION
df_data["TARGETPOP"] = df_data.apply(
    lambda row: targetpop_mapping[row["TARGETPOP_DESCRIPTION"]] if pd.isna(row["TARGETPOP"]) and row["TARGETPOP_DESCRIPTION"] in targetpop_mapping else row["TARGETPOP"],
    axis=1
)

In [39]:
# Step 2: Fill remaining missing TARGETPOP based on AGEADMINISTERED (if a mapping exists)
df_data["TARGETPOP"] = df_data.apply(
    lambda row: age_mapping[row["AGEADMINISTERED"]] if pd.isna(row["TARGETPOP"]) and row["AGEADMINISTERED"] in age_mapping else row["TARGETPOP"],
    axis=1
)   

In [40]:
# Check how many missing values remain
df_data["TARGETPOP"].isna().sum()

np.int64(1)

In [41]:
# Drop the row where TARGETPOP is still missing
df_data_cleaned = df_data.dropna(subset=["TARGETPOP"])


In [42]:
# Verify that missing values are fully handled
df_data_cleaned["TARGETPOP"].isna().sum(), len(df_data_cleaned)

(np.int64(0), 8052)

In [43]:
# Step 1: Identify missing values
missing_sourcecomment = df_data_cleaned["SOURCECOMMENT"].isna().sum()

In [44]:
# Step 2: Create mappings for imputation
sourcecomment_mapping_desc = df_data_cleaned.groupby("TARGETPOP_DESCRIPTION")["SOURCECOMMENT"].agg(
    lambda x: x.mode()[0] if not x.mode().empty else None
)

sourcecomment_mapping_age = df_data_cleaned.groupby("AGEADMINISTERED")["SOURCECOMMENT"].agg(
    lambda x: x.mode()[0] if not x.mode().empty else None
)

In [45]:
# Step 3: Impute missing values based on TARGETPOP_DESCRIPTION
df_data_cleaned["SOURCECOMMENT"] = df_data_cleaned.apply(
    lambda row: sourcecomment_mapping_desc[row["TARGETPOP_DESCRIPTION"]] 
    if pd.isna(row["SOURCECOMMENT"]) and row["TARGETPOP_DESCRIPTION"] in sourcecomment_mapping_desc 
    else row["SOURCECOMMENT"],
    axis=1
)

C:\Users\OMEN\AppData\Local\Temp\ipykernel_12868\3415126033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_cleaned["SOURCECOMMENT"] = df_data_cleaned.apply(


In [46]:
# Step 4: Impute remaining missing values based on AGEADMINISTERED
df_data_cleaned["SOURCECOMMENT"] = df_data_cleaned.apply(
    lambda row: sourcecomment_mapping_age[row["AGEADMINISTERED"]] 
    if pd.isna(row["SOURCECOMMENT"]) and row["AGEADMINISTERED"] in sourcecomment_mapping_age 
    else row["SOURCECOMMENT"],
    axis=1
)


C:\Users\OMEN\AppData\Local\Temp\ipykernel_12868\199897407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_cleaned["SOURCECOMMENT"] = df_data_cleaned.apply(


In [50]:
# Step 5: Save the updated dataset
updated_file_path = r"C:\Users\OMEN\OneDrive\Desktop\vaccine-schedule-data-cleaned.xlsx"
df_data_cleaned.to_excel(updated_file_path, sheet_name="Updated Data", index=False)

print("Updated dataset saved successfully!")

Updated dataset saved successfully!
